# Реализация основных алгоритмов машинного обучения 

#### 1. линейная регрессия (обучается стохастическим градиентым спуском+батчам)

In [1]:
class SGDLinearRegression:
    def __init__(self, step_size, epochs, batch_size, reg_weight):
        self.step_size = step_size   # шаг
        self.epochs = epochs         # число эпох 
        self.batch_size = batch_size # размер батча
        self.reg_weight = reg_weight # регуляризация
    
    def fit(self, train_features, train_target):
        
        # добавляем единичный вектор к признакам (свободный член)
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)   
        # целевой признак
        y = train_target
        # веса
        w = np.zeros(X.shape[1])
        
        # повторяем epochs раз
        for _ in range(self.epochs):
            
            # число батчей
            batches_count = X.shape[0] // self.batch_size
            
            # проходим по батчам
            for i in range(batches_count):
                # начало, конец
                begin = i * self.batch_size
                end = (i + 1) * self.batch_size                
                # срезаем данные
                X_batch = X[begin:end, :]
                y_batch = y[begin:end]
                
                # считаем градиент
                gradient = 2 * X_batch.T.dot(X_batch.dot(w) - y_batch) / X_batch.shape[0]
                
                # добавляем штраф
                reg = 2 * w.copy()
                reg[0]=0
                penalty = self.reg_weight * reg                
                gradient += penalty
                
                # обновляем веса
                w -= self.step_size * gradient

        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0


#### 2. к-ближайших соседей для регрессии

In [7]:
def knn_regression(X, y, n_neighbors, distance_func):
    # предсказания
    predictions = []    
    # проходим по строкам
    for i in range(X.shape[0]):        
        # текущая строка
        row = X[i]        
        # остальный строки
        other_rows = np.delete(X, i, axis=0)        
        # находим к ближайших строк к текущей и усредняем таргет для этих строк
        y_pred = np.mean(y[np.argsort([distance_func(row, other_row) for other_row in other_rows])[:n_neighbors]])        
        predictions.append(y_pred)    
    return predictions